In [1]:
# Download data
!wget https://files.webis.de/data-in-progress/data-research/web-search/reneuir-24/msmarco-passage-trec-dl-2019-judged-20230107-training-rerank.jsonl.gz

--2024-05-05 07:02:11--  https://files.webis.de/data-in-progress/data-research/web-search/reneuir-24/msmarco-passage-trec-dl-2019-judged-20230107-training-rerank.jsonl.gz
Resolving files.webis.de (files.webis.de)... 141.54.132.200
Connecting to files.webis.de (files.webis.de)|141.54.132.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6153798 (5.9M) [application/octet-stream]
Saving to: ‘msmarco-passage-trec-dl-2019-judged-20230107-training-rerank.jsonl.gz’

msmarco-passage-tre 100%[===================>]   5.87M  4.71MB/s    in 1.2s    

2024-05-05 07:02:13 (4.71 MB/s) - ‘msmarco-passage-trec-dl-2019-judged-20230107-training-rerank.jsonl.gz’ saved [6153798/6153798]



In [ ]:
!pip3 install ir_datasets python-terrier datasets sentence-transformers tira

In [9]:
import pandas as pd
from tira.third_party_integrations import normalize_run
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoModel, AutoModelForSequenceClassification
import numpy as np
from torch.utils.data import DataLoader
from sentence_transformers import CrossEncoder
import torch
from sentence_transformers import InputExample
import sentence_transformers
from tqdm import tqdm

In [10]:
import pyterrier as pt
import pandas as pd

if not pt.started():
  pt.init()

dataset_id = 'msmarco-passage/trec-dl-2019/judged'

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [11]:
initial_run = pd.read_json('msmarco-passage-trec-dl-2019-judged-20230107-training-rerank.jsonl.gz', lines=True, dtype={'qid': str, 'docno': str})
pt_dataset = pt.get_dataset(f'irds:{dataset_id}')
pt_bm25 = pt.transformer.get_transformer(initial_run)

In [15]:
models = [
    "Integer-Ctrl/cross-encoder-bert-tiny-512", "Integer-Ctrl/cross-encoder-bert-tiny-5120",
    "Integer-Ctrl/cross-encoder-bert-tiny-51200", "Integer-Ctrl/cross-encoder-bert-tiny-bs-200",
    "Integer-Ctrl/cross-encoder-bert-tiny-512"
]

for m in models:
  #Shortly ensure that all models are downloaded
  CrossEncoder(m, num_labels=1)

In [35]:
def make_predictions(model_name, dataset):
    ret = dataset.copy()
    input_data = []
    model =  CrossEncoder(model_name, num_labels=1)
    for _, i in ret.iterrows():

        input_data += [[i['query'], i['text']]]
    ret['score'] = model.predict(input_data)
    ret = normalize_run(ret, model_name)
    ret['qid'] = ret['qid'].astype(str)
    return pt.transformer.get_transformer(ret)

In [39]:
new_runs = []
new_run_names = []

for m in tqdm(models):
    new_runs += [make_predictions(m, initial_run)]
    new_run_names += [m]

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
 20%|██        | 1/5 [00:27<01:51, 27.97s/it]/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
 40%|████      | 2/5 [00:59<01:29, 29.78s/it]/usr/local/lib/python3.

In [41]:
pt.Experiment(
    [pt_bm25] + new_runs,
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank"],
    names=["BM25"] + new_run_names
)

,name,ndcg_cut.10,recip_rank
0,BM25,0.479540,0.795028
1,Integer-Ctrl/cross-encoder-bert-tiny-512,0.331722,0.548924
2,Integer-Ctrl/cross-encoder-bert-tiny-5120,0.475535,0.748345
3,Integer-Ctrl/cross-encoder-bert-tiny-51200,0.523269,0.853039
4,Integer-Ctrl/cross-encoder-bert-tiny-bs-200,0.621444,0.903277
5,Integer-Ctrl/cross-encoder-bert-tiny-512,0.331722,0.548924
